In [1]:
import torch
import pandas as pd
import numpy as np
import random
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix
from torch.utils.data import Dataset, DataLoader
from transformers import (
    AutoTokenizer, AutoModelForSequenceClassification,
    AutoModel, AdamW, get_scheduler, set_seed
)
from torch.nn import functional as F
from torch.cuda.amp import GradScaler, autocast
from tqdm import tqdm
from sklearn.model_selection import train_test_split, KFold
from scipy.stats import mannwhitneyu
from sklearn.model_selection import StratifiedKFold

# Seed setup
def seed_everything(seed=6):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

seed_everything(6)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Dataset
# class TextDataset(Dataset):
#     def __init__(self, texts, labels, tokenizer, max_len=128):
#         self.texts = texts
#         self.labels = labels
#         self.tokenizer = tokenizer
#         self.max_len = max_len

#     def __len__(self):
#         return len(self.texts)

#     def __getitem__(self, idx):
#         encoding = self.tokenizer.encode_plus(
#             self.texts[idx],
#             add_special_tokens=True,
#             max_length=self.max_len,
#             padding="max_length",
#             truncation=True,
#             return_attention_mask=True,
#             return_tensors="pt",
#         )
#         return {
#             "input_ids": encoding["input_ids"].squeeze(0),
#             "attention_mask": encoding["attention_mask"].squeeze(0),
#             "label": torch.tensor(self.labels[idx], dtype=torch.long),
#         }
class TextDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_len=512, overlap=128, cases=None):
        self.samples = []

        for idx, (text, label) in enumerate(zip(texts, labels)):
            case_id = cases[idx] if cases is not None else idx
            encoding = tokenizer.encode_plus(
                text,
                add_special_tokens=True,
                return_attention_mask=False,
                return_tensors=None,
            )
            input_ids = encoding["input_ids"]
            
            # Chunking
            start = 0
            while start < len(input_ids):
                end = start + max_len
                chunk = input_ids[start:end]
                
                if len(chunk) < max_len:
                    chunk += [tokenizer.pad_token_id] * (max_len - len(chunk))
                
                self.samples.append({
                    "input_ids": chunk,
                    "label": label,
                    "case_id": case_id,
                    "chunk_id": start // (max_len - overlap)  # optional: which chunk in that case
                })

                if end >= len(input_ids):
                    break
                start += max_len - overlap

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        sample = self.samples[idx]
        input_ids = torch.tensor(sample["input_ids"], dtype=torch.long)
        attention_mask = (input_ids != 0).long()  # assuming pad_token_id = 0
        return {
            "input_ids": input_ids,
            "attention_mask": attention_mask,
            "label": torch.tensor(sample["label"], dtype=torch.long),
            "case_id": sample["case_id"],
            "chunk_id": sample["chunk_id"],  # 可用于后续组合分析
        }


# Fallback for models without classification head
class CustomClassifier(torch.nn.Module):
    def __init__(self, base_model, hidden_size, num_labels=2):
        super().__init__()
        self.base = base_model
        self.dropout = torch.nn.Dropout(0.3)
        self.classifier = torch.nn.Linear(hidden_size, num_labels)

    def forward(self, input_ids, attention_mask, labels=None):
        outputs = self.base(input_ids=input_ids, attention_mask=attention_mask)
        pooled_output = outputs.last_hidden_state[:, 0]  # CLS
        logits = self.classifier(self.dropout(pooled_output))
        loss = None
        if labels is not None:
            loss = F.cross_entropy(logits, labels)
        return type('Output', (), {'loss': loss, 'logits': logits})
def dataset_to_dataframe(dataset, tokenizer):
    records = []
    for sample in dataset:
        input_ids = sample["input_ids"]
        text = tokenizer.decode(input_ids, skip_special_tokens=True)
        records.append({
            "case_id": sample["case_id"],
            "chunk_id": sample["chunk_id"],
            "label": sample["label"].item(),
            "chunk_text": text
        })
    return pd.DataFrame(records)
# Metrics
from sklearn.metrics import (
    roc_auc_score, f1_score, accuracy_score,
    precision_score, recall_score, confusion_matrix
)
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_predict
import pandas as pd
import numpy as np

def compute_metrics(y_true, y_prob, y_pred):
    auc = roc_auc_score(y_true, y_prob)
    f1 = f1_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred)
    recall = recall_score(y_true, y_pred)
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
    specificity = tn / (tn + fp)
    npv = tn / (tn + fn) if (tn + fn) > 0 else 0.0
    return {
        "AUC": auc,
        "F1": f1,
        "Sensitivity": recall,
        "Specificity": specificity,
        "PPV": precision,
        "NPV": npv
    }

def evaluate_case_level(fold_results):
    results = {}

    # 1. Majority Voting
    voting_df = (
        fold_results
        .assign(pred=(fold_results["prob1"] > 0.5).astype(int))
        .groupby("cased")
        .agg({
            "true_label": "first",
            "pred": lambda x: x.value_counts().idxmax()
        })
        .reset_index()
    )
    y_true = voting_df["true_label"]
    y_pred = voting_df["pred"]
    # For AUC we average prob1 by case
    prob_df = fold_results.groupby("cased").agg({"prob1": "mean"}).reset_index()
    y_prob = prob_df["prob1"]
    results["Majority Voting"] = compute_metrics(y_true, y_prob, y_pred)

    # 2. Soft Voting (average prob)
    soft_df = (
        fold_results
        .groupby("cased")
        .agg({
            "true_label": "first",
            "prob1": "mean"
        })
        .reset_index()
    )
    y_true = soft_df["true_label"]
    y_prob = soft_df["prob1"]
    y_pred = (y_prob > 0.5).astype(int)
    results["Soft Voting"] = compute_metrics(y_true, y_prob, y_pred)

    # 3. Stacking (Logistic Regression)
    stack_df = (
        fold_results
        .groupby("cased")
        .agg({
            "true_label": "first",
            "prob0": "mean",
            "prob1": "mean"
        })
        .reset_index()
    )
    X = stack_df[["prob0", "prob1"]]
    y = stack_df["true_label"]

    # 5-fold CV predicted probabilities
    clf = LogisticRegression()
    y_prob = cross_val_predict(clf, X, y, cv=5, method="predict_proba")[:, 1]
    y_pred = (y_prob > 0.5).astype(int)
    results["Stacking (LR)"] = compute_metrics(y, y_prob, y_pred)

    # Print all results
    for method, metrics in results.items():
        print(f"\n=== {method} ===")
        for k, v in metrics.items():
            print(f"{k}: {v:.4f}")


# Training
def train_epoch(model, data_loader, optimizer, scheduler, device, scaler):
    model.train()
    losses = []
    correct_predictions = 0

    for batch in tqdm(data_loader, desc="Training"):
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["label"].to(device)

        optimizer.zero_grad()

        with autocast():
            outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss
            logits = outputs.logits

        _, preds = torch.max(logits, dim=1)
        correct_predictions += torch.sum(preds == labels)
        losses.append(loss.item())

        scaler.scale(loss).backward()
        scaler.unscale_(optimizer)
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        scaler.step(optimizer)
        scaler.update()
        scheduler.step()

    return correct_predictions.double() / len(data_loader.dataset), np.mean(losses)

# Evaluation
def eval_model(model, data_loader, device):
    model.eval()
    true_labels, pred_probs, cased = [], [], []

    with torch.no_grad():
        for batch in tqdm(data_loader, desc="Validation"):
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            labels = batch["label"].to(device)

            outputs = model(input_ids, attention_mask=attention_mask)
            probs = F.softmax(outputs.logits, dim=1)

            true_labels.extend(labels.cpu().numpy())
            pred_probs.extend(probs.cpu().numpy())
            cased.extend(input_ids.cpu().numpy())

    return np.array(cased), np.array(true_labels), np.array(pred_probs)

# Model list
# namess = ['deepseekr1','Cluade-3.7','qwen','chatgpt_4o','gemini1.5_pro','LLama3.3']
models = [
    'ckiplab/albert-base-chinese',
    'indiejoseph/bert-base-cantonese',
    'zwzzz/Chinese-MentalBERT',
    'Geotrend/distilbert-base-zh-cased',
    'hfl/chinese-roberta-wwm-ext',
    'hfl/chinese-xlnet-base',
    'hfl/chinese-electra-base-discriminator'
]

names=['text','interviewee_text','new_text','new_interviewee_text']

all_model_metrics = []

for name in names:
    originaldata = pd.read_csv("combine.csv")
    texts = originaldata[name].tolist()
    labels = originaldata["label"].tolist()
    cases = originaldata["case"].tolist()

    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

    for model_name in models:
        tokenizer = AutoTokenizer.from_pretrained(model_name)
        result_all_folds = pd.DataFrame()

        for fold_idx, (train_index, val_index) in enumerate(skf.split(texts, labels)):
            train_texts = [texts[i] for i in train_index]
            train_labels = [labels[i] for i in train_index]

            val_texts = [texts[i] for i in val_index]
            val_labels = [labels[i] for i in val_index]
            val_cases = [cases[i] for i in val_index]

            train_dataset = TextDataset(train_texts, train_labels, tokenizer)
            val_dataset = TextDataset(val_texts, val_labels, tokenizer, cases=val_cases)

            train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True, generator=torch.Generator().manual_seed(6))
            val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False)

            try:
                model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2).to(device)
            except:
                base = AutoModel.from_pretrained(model_name)
                model = CustomClassifier(base_model=base, hidden_size=base.config.hidden_size).to(device)

            optimizer = AdamW(model.parameters(), lr=1e-5, correct_bias=False)
            scheduler = get_scheduler("linear", optimizer=optimizer, num_warmup_steps=0, num_training_steps=len(train_loader) * 5)
            scaler = GradScaler()

            for epoch in range(20):
                print(f"Model: {model_name}, Fold: {fold_idx + 1}, Epoch {epoch + 1}")
                train_acc, train_loss = train_epoch(model, train_loader, optimizer, scheduler, device, scaler)
                print(f"Train loss: {train_loss:.4f}, Accuracy: {train_acc:.4f}")

            # Evaluation
            cased, true_labels, pred_probs = eval_model(model, val_loader, device)
            val_chunks = dataset_to_dataframe(val_dataset, tokenizer)

            fold_results = pd.DataFrame({
                "cased": val_chunks["case_id"],
                "true_label": val_chunks["label"],
                "prob0": pred_probs[:, 0],
                "prob1": pred_probs[:, 1],
            })

            result_all_folds = pd.concat([result_all_folds, fold_results], ignore_index=True)

        # Majority Voting
        voting_df = (
            result_all_folds
            .assign(pred=(result_all_folds["prob1"] > 0.5).astype(int))
            .groupby("cased")
            .agg({
                "true_label": "first",
                "pred": lambda x: x.value_counts().idxmax(),
                "prob1": "mean"
            })
            .reset_index()
            .rename(columns={"cased": "case", "true_label": "label", "prob1": "prob"})
        )

        y_true = voting_df["label"]
        y_pred = voting_df["pred"]
        y_prob = voting_df["prob"]
        y_case = voting_df["case"]
        metrics = compute_metrics(y_true, y_prob, y_pred)
        model_short = model_name.split("/")[-1]
        print(f"\n=== {model_short} on {name} ===")
        for k, v in metrics.items():
            print(f"{k}: {v:.4f}")

        metrics["Model"] = model_short
        metrics["TextType"] = name
        all_model_metrics.append(metrics)

        # Save majority voting results
#         voting_df.to_csv(f"./newresult/{name}_{model_short}_majority_voting_results.csv", index=False)

# # Save all metrics
# metrics_df = pd.DataFrame(all_model_metrics)
# metrics_df.to_csv("./all_model_metrics_summary.csv", index=False)

Using device: cuda


Token indices sequence length is longer than the specified maximum sequence length for this model (1546 > 512). Running this sequence through the model will result in indexing errors
Some weights of AlbertForSequenceClassification were not initialized from the model checkpoint at ckiplab/albert-base-chinese and are newly initialized: ['albert.pooler.bias', 'albert.pooler.weight', 'classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
c:\Users\xianl\AppData\Local\Programs\Python\Python310\lib\site-packages\transformers\optimization.py:640: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
C:\Users\xianl\AppData\Local\Temp\ipykernel_16420\2001217518.py:326: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is 

Model: ckiplab/albert-base-chinese, Fold: 1, Epoch 1


Training:   0%|          | 0/94 [00:00<?, ?it/s]C:\Users\xianl\AppData\Local\Temp\ipykernel_16420\2001217518.py:238: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():
Training: 100%|██████████| 94/94 [00:21<00:00,  4.36it/s]


Train loss: 0.5747, Accuracy: 0.7253
Model: ckiplab/albert-base-chinese, Fold: 1, Epoch 2


...

Train loss: 0.5067, Accuracy: 0.7543
Model: zwzzz/Chinese-MentalBERT, Fold: 2, Epoch 8


Training: 100%|██████████| 95/95 [00:25<00:00,  3.69it/s]


Train loss: 0.5002, Accuracy: 0.7610
Model: zwzzz/Chinese-MentalBERT, Fold: 2, Epoch 9


Training:  63%|██████▎   | 60/95 [00:16<00:09,  3.64it/s]


KeyboardInterrupt: 

In [18]:
import torch
import pandas as pd
import numpy as np
import random
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix
from torch.utils.data import Dataset, DataLoader
from transformers import (
    AutoTokenizer, AutoModelForSequenceClassification,
    AutoModel, AdamW, get_scheduler, set_seed
)
from torch.nn import functional as F
from torch.cuda.amp import GradScaler, autocast
from tqdm import tqdm
from sklearn.model_selection import StratifiedKFold
from opencc import OpenCC
import jieba

# --- Seed setup ---
def seed_everything(seed=6):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

seed_everything(6)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

def tokenize_cantonese(text):
    return list(jieba.cut(text))
# --- Preprocessing function using Weikit and OpenCC ---
def preprocess_cantonese(texts, opencc_config='hk2s'):
    cc = OpenCC(opencc_config)
    processed = []
    for text in texts:
        text_conv = cc.convert(text)  # Convert to Mandarin/Simplified
        tokens = tokenize_cantonese(text_conv)  # Tokenize (still useful for segmentation)
        processed.append(" ".join(tokens))
    return processed

# --- Dataset ---
class TextDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_len=512, overlap=128, cases=None, pad_token_id=None):
        self.samples = []
        pad_token_id = pad_token_id if pad_token_id is not None else tokenizer.pad_token_id or 0
        for idx, (text, label) in enumerate(zip(texts, labels)):
            case_id = cases[idx] if cases is not None else idx
            encoding = tokenizer.encode_plus(
                text,
                add_special_tokens=True,
                return_attention_mask=False,
                return_tensors=None,
            )
            input_ids = encoding["input_ids"]
            # Chunking
            start = 0
            while start < len(input_ids):
                end = start + max_len
                chunk = input_ids[start:end]
                if len(chunk) < max_len:
                    chunk += [pad_token_id] * (max_len - len(chunk))
                self.samples.append({
                    "input_ids": chunk,
                    "label": label,
                    "case_id": case_id,
                    "chunk_id": start // (max_len - overlap)
                })
                if end >= len(input_ids):
                    break
                start += max_len - overlap

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        sample = self.samples[idx]
        input_ids = torch.tensor(sample["input_ids"], dtype=torch.long)
        attention_mask = (input_ids != 0).long()  # assumes pad_token_id is 0
        return {
            "input_ids": input_ids,
            "attention_mask": attention_mask,
            "label": torch.tensor(sample["label"], dtype=torch.long),
            "case_id": sample["case_id"],
            "chunk_id": sample["chunk_id"],
        }

# --- Model fallback if classification head is missing ---
class CustomClassifier(torch.nn.Module):
    def __init__(self, base_model, hidden_size, num_labels=2):
        super().__init__()
        self.base = base_model
        self.dropout = torch.nn.Dropout(0.3)
        self.classifier = torch.nn.Linear(hidden_size, num_labels)

    def forward(self, input_ids, attention_mask, labels=None):
        outputs = self.base(input_ids=input_ids, attention_mask=attention_mask)
        pooled_output = outputs.last_hidden_state[:, 0]  # CLS token
        logits = self.classifier(self.dropout(pooled_output))
        loss = None
        if labels is not None:
            loss = F.cross_entropy(logits, labels)
        return type('Output', (), {'loss': loss, 'logits': logits})

def dataset_to_dataframe(dataset, tokenizer):
    records = []
    for sample in dataset:
        input_ids = sample["input_ids"]
        text = tokenizer.decode(input_ids, skip_special_tokens=True)
        records.append({
            "case_id": sample["case_id"],
            "chunk_id": sample["chunk_id"],
            "label": sample["label"].item(),
            "chunk_text": text
        })
    return pd.DataFrame(records)

# --- Metrics ---
def compute_metrics(y_true, y_prob, y_pred):
    auc = roc_auc_score(y_true, y_prob)
    f1 = f1_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred)
    recall = recall_score(y_true, y_pred)
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
    specificity = tn / (tn + fp)
    npv = tn / (tn + fn) if (tn + fn) > 0 else 0.0
    return {
        "AUC": auc,
        "F1": f1,
        "Sensitivity": recall,
        "Specificity": specificity,
        "PPV": precision,
        "NPV": npv
    }

# --- Training ---
def train_epoch(model, data_loader, optimizer, scheduler, device, scaler):
    model.train()
    losses = []
    correct_predictions = 0

    for batch in tqdm(data_loader, desc="Training"):
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["label"].to(device)

        optimizer.zero_grad()

        with autocast():
            outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss
            logits = outputs.logits

        _, preds = torch.max(logits, dim=1)
        correct_predictions += torch.sum(preds == labels)
        losses.append(loss.item())

        scaler.scale(loss).backward()
        scaler.unscale_(optimizer)
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        scaler.step(optimizer)
        scaler.update()
        scheduler.step()

    return correct_predictions.double() / len(data_loader.dataset), np.mean(losses)

# --- Evaluation ---
def eval_model(model, data_loader, device):
    model.eval()
    true_labels, pred_probs, cased = [], [], []

    with torch.no_grad():
        for batch in tqdm(data_loader, desc="Validation"):
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            labels = batch["label"].to(device)

            outputs = model(input_ids, attention_mask=attention_mask)
            probs = F.softmax(outputs.logits, dim=1)

            true_labels.extend(labels.cpu().numpy())
            pred_probs.extend(probs.cpu().numpy())
            # FIX HERE:
            cased.extend(batch["case_id"])  # No .cpu().numpy() needed
    return np.array(cased), np.array(true_labels), np.array(pred_probs)

# --- Model list ---
models = [
    'ckiplab/albert-base-chinese',
    # 'indiejoseph/bert-base-cantonese',
    # 'zwzzz/Chinese-MentalBERT',
    # 'Geotrend/distilbert-base-zh-cased',
    # 'hfl/chinese-roberta-wwm-ext',
    # 'hfl/chinese-xlnet-base',
    # 'hfl/chinese-electra-base-discriminator'
]

names=['text','interviewee_text','new_text','new_interviewee_text']

all_model_metrics = []

for name in names:
    originaldata = pd.read_csv("combine.csv")
    raw_texts = originaldata[name].tolist()
    labels = originaldata["label"].tolist()
    cases = originaldata["case"].tolist()

    # --- Preprocess using Weikit + OpenCC for Cantonese ---
    texts = preprocess_cantonese(raw_texts, opencc_config='hk2s')  # or 't2s', depending on your text

    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

    for model_name in models:
        tokenizer = AutoTokenizer.from_pretrained(model_name)
        result_all_folds = pd.DataFrame()

        for fold_idx, (train_index, val_index) in enumerate(skf.split(texts, labels)):
            train_texts = [texts[i] for i in train_index]
            train_labels = [labels[i] for i in train_index]
            val_texts = [texts[i] for i in val_index]
            val_labels = [labels[i] for i in val_index]
            val_cases = [cases[i] for i in val_index]

            train_dataset = TextDataset(train_texts, train_labels, tokenizer)
            val_dataset = TextDataset(val_texts, val_labels, tokenizer, cases=val_cases)

            train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True, generator=torch.Generator().manual_seed(6))
            val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False)

            try:
                model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2).to(device)
            except:
                base = AutoModel.from_pretrained(model_name)
                model = CustomClassifier(base_model=base, hidden_size=base.config.hidden_size).to(device)

            optimizer = AdamW(model.parameters(), lr=1e-5, correct_bias=False)
            scheduler = get_scheduler("linear", optimizer=optimizer, num_warmup_steps=0, num_training_steps=len(train_loader) * 5)
            scaler = GradScaler()

            for epoch in range(5):  # Reduce epochs for speed; adjust as needed
                print(f"Model: {model_name}, Fold: {fold_idx + 1}, Epoch {epoch + 1}")
                train_acc, train_loss = train_epoch(model, train_loader, optimizer, scheduler, device, scaler)
                print(f"Train loss: {train_loss:.4f}, Accuracy: {train_acc:.4f}")

            # Evaluation
            cased, true_labels, pred_probs = eval_model(model, val_loader, device)
            val_chunks = dataset_to_dataframe(val_dataset, tokenizer)

            fold_results = pd.DataFrame({
                "cased": val_chunks["case_id"],
                "true_label": val_chunks["label"],
                "prob0": pred_probs[:, 0],
                "prob1": pred_probs[:, 1],
            })

            result_all_folds = pd.concat([result_all_folds, fold_results], ignore_index=True)

        # Majority Voting
        voting_df = (
            result_all_folds
            .assign(pred=(result_all_folds["prob1"] > 0.5).astype(int))
            .groupby("cased")
            .agg({
                "true_label": "first",
                "pred": lambda x: x.value_counts().idxmax(),
                "prob1": "mean"
            })
            .reset_index()
            .rename(columns={"cased": "case", "true_label": "label", "prob1": "prob"})
        )

        y_true = voting_df["label"]
        y_pred = voting_df["pred"]
        y_prob = voting_df["prob"]
        y_case = voting_df["case"]
        metrics = compute_metrics(y_true, y_prob, y_pred)
        model_short = model_name.split("/")[-1]
        print(f"\n=== {model_short} on {name} ===")
        for k, v in metrics.items():
            print(f"{k}: {v:.4f}")

        metrics["Model"] = model_short
        metrics["TextType"] = name
        all_model_metrics.append(metrics)

        # Optionally save results for each model
        # voting_df.to_csv(f"./newresult/{name}_{model_short}_majority_voting_results.csv", index=False)

# # Save all metrics summary
# metrics_df = pd.DataFrame(all_model_metrics)
# metrics_df.to_csv("./all_model_metrics_summary.csv", index=False)

Using device: cuda


Token indices sequence length is longer than the specified maximum sequence length for this model (1546 > 512). Running this sequence through the model will result in indexing errors
Some weights of AlbertForSequenceClassification were not initialized from the model checkpoint at ckiplab/albert-base-chinese and are newly initialized: ['albert.pooler.bias', 'albert.pooler.weight', 'classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
c:\Users\xianl\AppData\Local\Programs\Python\Python310\lib\site-packages\transformers\optimization.py:640: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
C:\Users\xianl\AppData\Local\Temp\ipykernel_16420\2744621110.py:239: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is 

Model: ckiplab/albert-base-chinese, Fold: 1, Epoch 1


Training:   0%|          | 0/94 [00:00<?, ?it/s]C:\Users\xianl\AppData\Local\Temp\ipykernel_16420\2744621110.py:150: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():
Training: 100%|██████████| 94/94 [00:47<00:00,  1.99it/s]


Train loss: 0.5848, Accuracy: 0.7360
Model: ckiplab/albert-base-chinese, Fold: 1, Epoch 2


...

Train loss: 0.4402, Accuracy: 0.7932
Model: ckiplab/albert-base-chinese, Fold: 5, Epoch 5


Training: 100%|██████████| 42/42 [00:22<00:00,  1.83it/s]


Train loss: 0.3436, Accuracy: 0.8586


Validation: 100%|██████████| 11/11 [00:02<00:00,  5.07it/s]


=== albert-base-chinese on new_interviewee_text ===
AUC: 0.6537
F1: 0.7748
Sensitivity: 0.9053
Specificity: 0.2477
PPV: 0.6772
NPV: 0.6000


In [17]:
def compute_metrics(y_true, y_prob, y_pred):
    auc = roc_auc_score(y_true, y_prob)
    f1 = f1_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred)
    recall = recall_score(y_true, y_pred)
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
    specificity = tn / (tn + fp)
    npv = tn / (tn + fn) if (tn + fn) > 0 else 0.0
    return {
        "AUC": auc,
        "F1": f1,
        "Sensitivity": recall,
        "Specificity": specificity,
        "PPV": precision,
        "NPV": npv
    }
data=pd.read_csv('./newresult/text_albert-base-chinese_majority_voting_results.csv')
compute_metrics(data['label'], data['prob'], data['pred'])


{'AUC': np.float64(0.7530661516175761),
 'F1': 0.7903930131004366,
 'Sensitivity': 0.9526315789473684,
 'Specificity': np.float64(0.2018348623853211),
 'PPV': 0.6753731343283582,
 'NPV': np.float64(0.7096774193548387)}